<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       DFFT2Conv Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> The DFFT2Conv() function is a Fast Fourier Transform converter that converts output generated by DFFT2(). DFFT2() results are in human-readable and raw form. DFFT2Conv() can also output different payload types such as AMPL_PHASE_DEGREES, MULTIVAR_AMPL_PHASE_RADIANS, COMPLEX. However, user may want a different result form in order to pass the results to another UAF function, plot the results in human readable form, compare the results to Python, and so on. Instead of rerunning DFFT2() to produce a different result set, user can use DFFT2Conv() to accomplish the task faster and with less memory consumption.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    TDMatrix,
    DFFT2,
    DFFT2Conv,
    TDAnalyticResult,
    remove_context
    )

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_DFFT2Conv_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["dfft2conv_real_4_4"])

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data = DataFrame.from_table("dfft2conv_real_4_4")
data

<p style = 'font-size:16px;font-family:Arial'>Since the DFFT2Conv functions uses the output of DFFT2 function, we will first use the DFFT2 function and than use the DFFT2 output as input to the DFFT2Conv function.</p>

<p style = 'font-size:16px;font-family:Arial'>To use the DFFT2 fubction we will first create teradataml TDMatrix object from the DataFrame.</p>

In [ ]:
td_matrix = TDMatrix(data=data,
                     id="id",
                     row_index="row_i",
                     row_index_style="SEQUENCE",
                     column_index="column_i",
                     column_index_style="SEQUENCE",
                     payload_field="magnitude",
                     payload_content="REAL")

<p style = 'font-size:16px;font-family:Arial'>Than using the TDMatrix output we will compute the two-dimension fourier transform with real numbers only for the matrix id 33.</p>

In [ ]:
filter_expr = td_matrix.id==33
dfft2_out = DFFT2(data=td_matrix,
                  data_filter_expr=filter_expr,
                  freq_style="K_INTEGRAL",
                  human_readable=False,
                  output_fmt_content="COMPLEX")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. DFFT2Conv</b>
<p style = 'font-size:16px;font-family:Arial'>We will convert the complex(REAL,IMAGINARY) output of DFFT2() to polar(AMPLITUDE,PHASE) in RADIAN format using TDMatrix as input data.</p>


In [ ]:
matrix_spec = TDMatrix(data=dfft2_out.result,
                           id="id",
                           row_index="ROW_I",
                           column_index="COLUMN_I",
                           row_index_style="SEQUENCE",
                           column_index_style="SEQUENCE",
                           payload_field=["REAL_magnitude", "IMAG_magnitude"],
                           payload_content="COMPLEX")

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(DFFT2Conv)

<p style = 'font-size:16px;font-family:Arial'><b>Input Parameters:</b></p>
<li style = 'font-size:16px;font-family:Arial'><code>data:</code> Specifies the input matrix or a TDAnalyticResult object created over the output generated by the UAF function.</li>
<li style = 'font-size:16px;font-family:Arial'><code>conv</code> Specifies the type of conversion to be performed by the function. The Permitted Values are: <b>HR_TO_RAW</b> and <b>RAW_TO_HR</b>.</li>
<li style = 'font-size:16px;font-family:Arial'><code>output_fmt_content:</code> Specifies a complex number that can be in rectangular (real, imaginary) or polar (amplitude, phase) form. The default is rectangular (real,imaginary).</li>

In [ ]:
uaf_out1 = DFFT2Conv(data=matrix_spec,
                         conv="HR_TO_RAW",
                         output_fmt_content="AMPL_PHASE_RADIANS")
 
# Print the result DataFrame.
uaf_out1.result

<p style = 'font-size:16px;font-family:Arial'>In this example we will convert the complex(REAL,IMAGINARY) output of DFFT2() to polar(AMPLITUDE,PHASE) in RADIAN format using TDAnalyticResult as input data.</p>
<p style = 'font-size:16px;font-family:Arial'>First, we create teradataml TDAnalyticResult object from output of DFFT2()</p>

In [ ]:
art_spec = TDAnalyticResult(data=dfft2_out.result)

uaf_out2 = DFFT2Conv(data=art_spec,
                     conv="HR_TO_RAW",
                     output_fmt_content="AMPL_PHASE_RADIANS")
 
# Print the result DataFrame.
uaf_out2.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>DFFT2Conv function reference: <a href = 'https://docs.teradata.com/search/all?query=DFFT2Conv&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>